In [1]:
import os
import pandas as pd
import json
from dotenv import load_dotenv

from neomodel import (
    StructuredNode,
    StringProperty,
    IntegerProperty,
    RelationshipTo,
)

load_dotenv()

True

In [2]:
URI=os.getenv("NEO4J_URI")
USERNAME=os.getenv("NEO4J_USERNAME")
PASSWORD=os.getenv("NEO4J_PASSWORD")

In [3]:
cabang_df = pd.read_excel("salon_graph_data.xlsx", sheet_name="Cabang")
service_df = pd.read_excel("salon_graph_data.xlsx", sheet_name="Service")
daerah_df = pd.read_excel("salon_graph_data.xlsx", sheet_name="Daerah")
employee_df = pd.read_excel("salon_graph_data.xlsx", sheet_name="Employee")

In [ ]:
class Daerah(StructuredNode):
    id_salon = IntegerProperty(unique_index=True)
    nama_daerah = StringProperty()
    kecamatan = StringProperty()
    kota = StringProperty()
    kode_pos = StringProperty()

class Cabang(StructuredNode):
    id_salon = IntegerProperty(unique_index=True)
    name = StringProperty()
    year = IntegerProperty()

    # Relationships
    is_located = RelationshipTo("Daerah", "IS_LOCATED")
    has_service = RelationshipTo("Service", "HAS_SERVICE")
    has_employee = RelationshipTo("Employee", "HAS_EMPLOYEE")

class Service(StructuredNode):
    id_service = StringProperty(unique_index=True)
    name = StringProperty()
    price = IntegerProperty()

class Employee(StructuredNode):
    id_employee = IntegerProperty(unique_index=True) 
    name = StringProperty()
    work_start_hour = StringProperty()
    work_end_hour = StringProperty()
    id_salon = IntegerProperty() 



In [5]:
for _, row in daerah_df.iterrows():
    if not Daerah.nodes.get_or_none(id_salon=row["id_salon"]):
        Daerah(
            id_salon=row["id_salon"],
            nama_daerah=row["nama_daerah"],
            kecamatan=row["kecamatan"],
            kota=row["kota"],
            kode_pos=row["kode_pos"]
        ).save()

service_map = {}
for _, row in service_df.iterrows():
    required_columns = ["id", "name", "price"]
    if all(col in service_df.columns for col in required_columns):
        service_id = row["id"]
        if not Service.nodes.get_or_none(id_service=service_id):
            service = Service(
                id_service=service_id,
                name=row["name"],
                price=row["price"]
            ).save()
            service_map[service_id] = service
    else:
        print(f"Skipping invalid product row: {row}")

for _, row in cabang_df.iterrows():
    cabang_id = row["id_salon"]
    if not Cabang.nodes.get_or_none(id_salon=cabang_id):
        Cabang(
            id_salon=cabang_id,
            name=row["salon_name"],
            year=row["tahun_berdiri"]
        ).save()

for _, row in employee_df.iterrows():
    employee_id = row["id_employee"]
    if not Employee.nodes.get_or_none(id_employee=employee_id):
        Employee(
            id_employee=employee_id,
            name=row["name"],
            work_start_hour=row["work_start_hour"],
            work_end_hour=row["work_end_hour"],
            id_salon=row["id_salon"]
        ).save()



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Daerah)} {position: line: 1, column: 16, offset: 15} for query: ' MATCH (daerah:Daerah) WHERE daerah.id_salon = $daerah_id_salon_1 RETURN daerah LIMIT 2'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing prope

In [6]:
for _, row in daerah_df.iterrows():
    id_salon_val = row["id_salon"]
    if pd.notna(id_salon_val) and not Daerah.nodes.get_or_none(id_salon=int(id_salon_val)):
        Daerah(
            id_salon=int(id_salon_val),
            nama_daerah=str(row["nama_daerah"]) if pd.notna(row["nama_daerah"]) else "",
            kecamatan=str(row["kecamatan"]) if pd.notna(row["kecamatan"]) else "",
            kota=str(row["kota"]) if pd.notna(row["kota"]) else "",
            kode_pos=str(row["kode_pos"]) if pd.notna(row["kode_pos"]) else ""
        ).save()

service_map = {}
for _, row in service_df.iterrows():
    required_columns = ["id", "name", "price"]
    if all(col in service_df.columns for col in required_columns):
        service_id = str(row["id"])
        if pd.notna(service_id) and not Service.nodes.get_or_none(id_service=service_id):
            service = Service(
                id_service=service_id,
                name=str(row["name"]) if pd.notna(row["name"]) else "",
                price=int(row["price"]) if pd.notna(row["price"]) else 0
            ).save()
            service_map[service_id] = service
    else:
        print(f"Skipping invalid service row: missing required columns")

for _, row in cabang_df.iterrows():
    cabang_id = row["id_salon"]
    if pd.notna(cabang_id) and not Cabang.nodes.get_or_none(id_salon=int(cabang_id)):
        Cabang(
            id_salon=int(cabang_id),
            name=str(row["salon_name"]) if pd.notna(row["salon_name"]) else "",
            year=int(row["tahun_berdiri"]) if pd.notna(row["tahun_berdiri"]) else 0
        ).save()

for _, row in employee_df.iterrows():
    employee_id = str(row["id_employee"])
    if pd.notna(employee_id) and not Employee.nodes.get_or_none(id_employee=employee_id):
        Employee(
            id_employee=employee_id,
            name=str(row["name"]) if pd.notna(row["name"]) else "",
            work_start_hour=str(row["work_start_hour"]) if pd.notna(row["work_start_hour"]) else "",
            work_end_hour=str(row["work_end_hour"]) if pd.notna(row["work_end_hour"]) else "",
            id_salon=int(row["id_salon"]) if pd.notna(row["id_salon"]) else None
        ).save()



In [7]:
for cabang in Cabang.nodes.all():
    if cabang.id_salon is not None:
        daerah_node = Daerah.nodes.get_or_none(id_salon=cabang.id_salon)
        if daerah_node and not cabang.is_located.is_connected(daerah_node):
            cabang.is_located.connect(daerah_node)
            print(f"Connected Cabang {cabang.id_salon} to Daerah")

for cabang in Cabang.nodes.all():
    if cabang.id_salon is not None:
        employees = Employee.nodes.filter(id_salon=cabang.id_salon)
        for emp in employees:
            if not cabang.has_employee.is_connected(emp):
                cabang.has_employee.connect(emp)
                print(f"Connected Cabang {cabang.id_salon} to Employee {emp.id_employee}")

all_services = Service.nodes.all()
for cabang in Cabang.nodes.all():
    for serv in all_services:
        if not cabang.has_service.is_connected(serv):  # Fixed: was has_product
            cabang.has_service.connect(serv)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: IS_LOCATED)} {position: line: 1, column: 81, offset: 80} for query: 'MATCH (cabang) WHERE elementId(cabang)=$cabang_1 WITH cabang MATCH (cabang)-[r1:`IS_LOCATED`]->(is_located_r1:Daerah) WHERE elementId(is_located_r1) = $is_located_r1_contains_1 WITH is_located_r1 RETURN count(is_located_r1)'


Connected Cabang 1 to Daerah
Connected Cabang 2 to Daerah
Connected Cabang 3 to Daerah
Connected Cabang 4 to Daerah
Connected Cabang 5 to Daerah
Connected Cabang 6 to Daerah
Connected Cabang 7 to Daerah
Connected Cabang 8 to Daerah


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: HAS_EMPLOYEE)} {position: line: 1, column: 81, offset: 80} for query: 'MATCH (cabang) WHERE elementId(cabang)=$cabang_1 WITH cabang MATCH (cabang)-[r1:`HAS_EMPLOYEE`]->(has_employee_r1:Employee) WHERE elementId(has_employee_r1) = $has_employee_r1_contains_1 WITH has_employee_r1 RETURN count(has_employee_r1)'


Connected Cabang 9 to Daerah
Connected Cabang 10 to Daerah
Connected Cabang 1 to Employee 1
Connected Cabang 1 to Employee 2
Connected Cabang 1 to Employee 3
Connected Cabang 1 to Employee 4
Connected Cabang 1 to Employee 5
Connected Cabang 1 to Employee 6
Connected Cabang 1 to Employee 7
Connected Cabang 1 to Employee 8
Connected Cabang 1 to Employee 9
Connected Cabang 1 to Employee 10
Connected Cabang 2 to Employee 11
Connected Cabang 2 to Employee 12
Connected Cabang 2 to Employee 13
Connected Cabang 2 to Employee 14
Connected Cabang 2 to Employee 15
Connected Cabang 2 to Employee 16
Connected Cabang 2 to Employee 17
Connected Cabang 2 to Employee 18
Connected Cabang 2 to Employee 19
Connected Cabang 2 to Employee 20
Connected Cabang 3 to Employee 21
Connected Cabang 3 to Employee 22
Connected Cabang 3 to Employee 23
Connected Cabang 3 to Employee 24
Connected Cabang 3 to Employee 25
Connected Cabang 3 to Employee 26
Connected Cabang 3 to Employee 27
Connected Cabang 3 to Employee 2

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: HAS_SERVICE)} {position: line: 1, column: 81, offset: 80} for query: 'MATCH (cabang) WHERE elementId(cabang)=$cabang_1 WITH cabang MATCH (cabang)-[r1:`HAS_SERVICE`]->(has_service_r1:Service) WHERE elementId(has_service_r1) = $has_service_r1_contains_1 WITH has_service_r1 RETURN count(has_service_r1)'
